<a href="https://www.kaggle.com/code/franciscosuarezb/crosswalk-detection?scriptVersionId=185981839" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

In [9]:
data_dir = '/kaggle/input/crosswalk-detection-dataset/Data'
images_dir = os.path.join(data_dir, 'images')
labels_dir = os.path.join(data_dir, 'labels')

In [10]:
# Listar archivos de imágenes y labels
image_files = os.listdir(images_dir)
label_files = os.listdir(labels_dir)

print(f"Total imágenes: {len(image_files)}")
print(f"Total labels: {len(label_files)}")


Total imágenes: 1523
Total labels: 1518


In [11]:
# Dividir en entrenamiento y prueba
train_images, test_images = train_test_split(image_files, test_size=0.15, random_state=42)
# Dividir el conjunto de entrenamiento en entrenamiento y validación
train_images, val_images = train_test_split(train_images, test_size=0.20, random_state=42)  # 0.25 * 0.8 = 0.2

print(f"Total entrenamiento: {len(train_images)}")
print(f"Total validación: {len(val_images)}")
print(f"Total prueba: {len(test_images)}")


Total entrenamiento: 1035
Total validación: 259
Total prueba: 229


In [12]:
output_dir = '/kaggle/working/dataset'

# Crear directorios si no existen
os.makedirs(os.path.join(output_dir, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val/labels'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test/labels'), exist_ok=True)


In [13]:
print(f'Train_images: {train_images[:3]}')
print(f'Train_images: {test_images[:3]}')
print(f'Train_images: {val_images[:3]}')

Train_images: ['Strisce--504-_jpg.rf.43a11bdc321bbc9b6a961048ce74bbe0.jpg', 'Strisce--84-_jpg.rf.7eae2d3d0599b9d2cbf498c6cbe6243f.jpg', 'Strisce--360-_jpg.rf.9227409c7f73d123e97ca7ab63659a2d.jpg']
Train_images: ['Strisce--338-_jpg.rf.504896b03879b76a3c88044b77c14022.jpg', 'Strisce--162-_jpg.rf.dae43a9d1a31c52968dd89c4974c3490.jpg', 'Strisce--342-_jpg.rf.7c988bcd55d6c4ab72082717f159cfa3.jpg']
Train_images: ['Strisce--258-_jpg.rf.f80257ec4f8882c6d922f01e78a71d55.jpg', 'Strisce--195-_jpg.rf.024f14a7cba5000bec899b903c56a133.jpg', 'Strisce--226-_jpg.rf.7572d0531ef340e7db84174abd8c0026.jpg']


In [14]:
# Crear una nueva lista con la extensión .txt
train_labels = [f.replace('.jpg', '.txt') for f in train_images]
val_labels=[f.replace('.jpg', '.txt') for f in val_images]
test_labels=[f.replace('.jpg', '.txt') for f in test_images]

print(f' Total Train_Labels:{len(train_labels)}')
print(f'Total Val_Labels:{len(val_labels)}')
print(f'Total test_labels:{len(test_labels)}')

 Total Train_Labels:1035
Total Val_Labels:259
Total test_labels:229


In [15]:
print(f'Train_Labels:{train_labels[:3]}')
print(f'Val_Labels:{val_labels[:3]}')
print(f'Test_Labels:{test_labels[:3]}')


Train_Labels:['Strisce--504-_jpg.rf.43a11bdc321bbc9b6a961048ce74bbe0.txt', 'Strisce--84-_jpg.rf.7eae2d3d0599b9d2cbf498c6cbe6243f.txt', 'Strisce--360-_jpg.rf.9227409c7f73d123e97ca7ab63659a2d.txt']
Val_Labels:['Strisce--258-_jpg.rf.f80257ec4f8882c6d922f01e78a71d55.txt', 'Strisce--195-_jpg.rf.024f14a7cba5000bec899b903c56a133.txt', 'Strisce--226-_jpg.rf.7572d0531ef340e7db84174abd8c0026.txt']
Test_Labels:['Strisce--338-_jpg.rf.504896b03879b76a3c88044b77c14022.txt', 'Strisce--162-_jpg.rf.dae43a9d1a31c52968dd89c4974c3490.txt', 'Strisce--342-_jpg.rf.7c988bcd55d6c4ab72082717f159cfa3.txt']


In [16]:
import os
import shutil

def move_files(file_list, src_dir, dst_dir):
    for file_name in file_list:
        src_file = os.path.join(src_dir, file_name)
        dst_file = os.path.join(dst_dir, file_name)
        try:
            shutil.copy(src_file, dst_file)
        except FileNotFoundError:
            print(f"Archivo no encontrado: {src_file}. Creando un archivo .txt vacío.")
            if file_name.endswith('.txt'):
                open(dst_file, 'a').close()

# Mover archivos de entrenamiento
move_files(train_images, images_dir, os.path.join(output_dir, 'train/images'))
move_files(train_labels, labels_dir, os.path.join(output_dir, 'train/labels'))

# Mover archivos de validación
move_files(val_images, images_dir, os.path.join(output_dir, 'val/images'))
move_files(val_labels, labels_dir, os.path.join(output_dir, 'val/labels'))

# Mover archivos de prueba
move_files(test_images, images_dir, os.path.join(output_dir, 'test/images'))
move_files(test_labels, labels_dir, os.path.join(output_dir, 'test/labels'))


Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--118-_jpg.rf.54995eb9bae14606e4cddbc834e8ce37(1).txt. Creando un archivo .txt vacío.
Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--105-_jpg.rf.16ab0e51e3a52d6520b0c449d2a8b56b(1).txt. Creando un archivo .txt vacío.
Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--113-_jpg.rf.1fcdddf1cf3d22ab22bf5b31bdd62a1e(1).txt. Creando un archivo .txt vacío.
Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--107-_jpg.rf.17c8ef02eb84b49a728055218dceaa40(1).txt. Creando un archivo .txt vacío.
Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--105-_jpg.rf.e2fce3b37caa1e8452748ffbadde64df(1).txt. Creando un archivo .txt vacío.


In [18]:
for phase in ['train', 'val', 'test']:
    print(f"{phase} images: {len(os.listdir(os.path.join(output_dir, phase, 'images')))}")
    print(f"{phase} labels: {len(os.listdir(os.path.join(output_dir, phase, 'labels')))}")


train images: 1035
train labels: 1035
val images: 259
val labels: 259
test images: 229
test labels: 229


In [19]:
from datetime import datetime
from functools import wraps

import numpy as np
from IPython.lib.display import Audio
from ultralytics import YOLO


# Crear el yaml archivo de conf de yolov8

In [32]:
yaml_content = """
path: /kaggle/working/dataset

train: train/images
val: val/images


nc: 1

names:
     - crosswalk
"""

# Guardar el archivo
with open('/kaggle/working/crosswalk_dataset.yaml', 'w') as file:
    file.write(yaml_content)

print("Archivo .yaml creado con éxito.")


Archivo .yaml creado con éxito.


# Crear una carpeta para guardar el proyecto

In [33]:
os.makedirs(os.path.join('/kaggle/working/', 'training_results'), exist_ok=True)


In [34]:
proyect_results_path="/kaggle/working/training_results"

## Cargar los pesos pre-entrenados

In [35]:
#%%
from ultralytics import YOLO

epochs = 150 
weight = 'yolov8n.pt'
batch = 64 

model = YOLO(weight)
data_training =  "/kaggle/working/crosswalk_dataset.yaml"
project = proyect_results_path


In [36]:
print(type(model))

<class 'ultralytics.models.yolo.model.YOLO'>


In [39]:
%pip install -U ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [37]:
 model.train(data=data_training,
             epochs=epochs,
             imgsz=512,
             batch=batch,
             cache=True,
             patience=20,
             project=project,
             device=[0, 1]  # Esto especifica el uso de las GPUs 0 y 1
            )

Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/crosswalk_dataset.yaml, epochs=150, time=None, patience=20, batch=32, imgsz=512, save=True, save_period=-1, cache=True, device=[0, 1], workers=8, project=/kaggle/working/training_results, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sh

100%|██████████| 755k/755k [00:00<00:00, 13.9MB/s]
2024-06-28 23:43:39,722	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-28 23:43:40,719	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/dataset/train/labels... 1035 images, 18 backgrounds, 0 corrupt: 100%|██████████| 1035/1035 [00:00<00:00, 1130.39it/s]


train: New cache created: /kaggle/working/dataset/train/labels.cache


...

                   all        259        392      0.843      0.725      0.864      0.751

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    150/150      1.57G      0.532     0.4913     0.9484          6        512: 100%|██████████| 33/33 [00:05<00:00,  5.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.81it/s]


                   all        259        392      0.842      0.732      0.867      0.759

150 epochs completed in 0.360 hours.
Optimizer stripped from /kaggle/working/training_results/train3/weights/last.pt, 6.2MB
Optimizer stripped from /kaggle/working/training_results/train3/weights/best.pt, 6.2MB

Validating /kaggle/working/training_results/train3/weights/best.pt...
Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.72it/s]


                   all        259        392      0.863      0.712      0.867      0.759
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /kaggle/working/training_results/train3
